In [1]:
import os
import time
import openpyxl
import pandas as pd
from typing import List
from loguru import logger
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc



load_dotenv()

True

In [2]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            logger.info('Conexão estabelecida!')
            pass
        logger.info('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        logger.info(f"Error: {e}")
        return None

In [ ]:
def generate_sql_query() -> List[str]:
    current_month = f"{datetime.now().month:02}"
    current_year = datetime.now().year % 100  # Get the last two digits of the year

    last_month = f"{(datetime.now().month - 1) % 12:02}"
    last_year = current_year if datetime.now().month > 1 else current_year - 1

    tables = [f"movprodd{last_month}{last_year}", f"movprodd{current_month}{current_year}"]

    return tables

print(generate_sql_query())

In [5]:
conn = get_db_engine() 

def vendas_query(conn):
    query = (f"""
        SELECT 
                mprd.mprd_transacao AS transacao,
                clie.clie_cnpjcpf AS cnpj_cpf,
                clie.clie_nome AS nome_clie,
                mprd.mprd_datamvto AS data_mvto,
                mprd.mprd_numerodcto AS nfe,
                prod.prod_codbarras AS cod_barras,
                prod.prod_pesoliq AS peso,
                mprd_prod_codigo::TEXT AS cod_prod,
                TO_CHAR(mprd.mprd_qtde, '00000000000999D9999') AS qtde,
                TO_CHAR((mprd.mprd_valor / mprd.mprd_qtde), '00999D99') AS valor_unitario_2,
                TO_CHAR(mprd.mprd_vlrunitario, '00999D99') AS valor_unitario,
                mprc.mprc_vend_codigo AS cod_vend,
                mprc.mprc_codentidade AS cod_clie,
                mprd.mprd_dcto_codigo AS dcto_cod,
                prun.prun_emb AS embalagem_vend,
                SUBSTRING(clie.clie_cepres, 1,5) ||'-'|| SUBSTRING(clie.clie_cepres, 6,3) AS cep
            FROM movprodd0722 AS mprd
            LEFT JOIN movprodc AS mprc ON mprd.mprd_transacao = mprc.mprc_transacao
            LEFT JOIN produtos AS prod ON mprd.mprd_prod_codigo = prod.prod_codigo
            LEFT JOIN clientes AS clie ON mprc.mprc_codentidade = clie.clie_codigo
            LEFT JOIN (
                SELECT prun_prod_codigo, prun_emb
                FROM produn
                WHERE prun_unid_codigo = '010'
                ORDER BY prun_prod_codigo, prun_emb
            ) AS prun ON mprd.mprd_prod_codigo = prun.prun_prod_codigo
            WHERE mprd_status = 'N'
            AND mprd_unid_codigo = '010'
            AND prod.prod_marca IN ('BRF', 'BRF IN NATURA')
            AND mprd.mprd_dcto_codigo IN ('6666','6668','7339','7335','7338','7337','7260','7263','7262','7268','7264','7269','7267','7319','7318', '6680','6890')
            AND mprc.mprc_vend_codigo NOT IN ('0','00','000','0000','')
            AND mprd.mprd_vlrunitario NOT IN ('0.0100000000')
            AND clie.clie_cepres NOT IN ('00000-000','','0','00000','00000000')
            AND clie.clie_cepres > '0'
            AND clie.clie_cepres NOT IN ('')
            AND prod.prod_codbarras = '7891515969509'
    """)
        # AND mprd.mprd_datamvto > '2023-01-09'
    
    df = pd.read_sql_query(query, conn)
    return df

display(vendas_query(conn))

2023-10-23 09:19:42.950 | INFO     | __main__:get_db_engine:7 - Conexão estabelecida!
2023-10-23 09:19:42.951 | INFO     | __main__:get_db_engine:9 - Banco de dados conectado!


,transacao,cnpj_cpf,nome_clie,data_mvto,nfe,cod_barras,peso,cod_prod,qtde,valor_unitario_2,valor_unitario,cod_vend,cod_clie,dcto_cod,embalagem_vend,cep
